In [1]:
import copy
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd

c:\users\user\pycharmprojects\tensorflowsudoku\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\pycharmprojects\tensorflowsudoku\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\user\pycharmprojects\tensorflowsudoku\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
def norm(array):
   return array/9

def invNorm(array):
   return 9*array

In [3]:
games = pd.read_csv("./data/sudoku.csv")

quizzes = games['quizzes']
solutions = games['solutions']

features = []
labels = []

for i in range(int(len(quizzes)/1)):
   temp_arr = [norm(int(char)) for char in quizzes[i]]
   features.append(np.array(temp_arr).reshape((9,9,1)))

   if(i%200000==0):
       print(str(i/20000) + "%")



#We represent a 1 as label 0, 2 as label 1, etc.

for i in range(int(len(solutions)/1)):
   temp_arr = [(int(char)-1) for char in solutions[i]]
   labels.append(np.array(temp_arr).reshape((81,1)))
   if(i%200000==0):
       print(str(i/20000 +50)+ "%")


features = np.array(features)
labels = np.array(labels)
print("100%")
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100%


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, kernel_size=(1,1), activation='relu', padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(81*9))
model.add(layers.Reshape((81, 9)))
model.add(layers.Activation('softmax'))
model.summary()

In [15]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=16, epochs=3, validation_data=(x_test, y_test))


Epoch 1/3
 4960/50000 [=>............................] - ETA: 19:22 - loss: 1.5687 - accuracy: 0.8016

KeyboardInterrupt: 

In [ ]:
model.save_weights('checkpoints/latest_checkpoint')




In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, kernel_size=(1,1), activation='relu', padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(81*9))
model.add(layers.Reshape((81, 9)))
model.add(layers.Activation('softmax'))
model.load_weights('./checkpoints/latest_checkpoint')


In [13]:
def step_by_step(inputGame):

    finished = False
    while not finished:

        predictions = model.predict(inputGame.reshape((1,9,9,1)))[0]
        #Unnormalize the predictions and square values
        predict = np.argmax(predictions, axis=1).reshape((9,9))+1
        prob = np.max(predictions, axis=1).reshape((9,9))
        inputGame = invNorm(inputGame).reshape((9,9))
        missing = (inputGame==0)

        if missing.sum()==0:
            finished = True
        else:
            max = 0
            ind = 0
            for i in range(9):
                for j in range(9):
                    if missing[i][j] and prob[i][j]>= max:
                        max = prob[i][j]
                        ind = i*9 + j
                    elif missing[i][j] and prob[i][j]>0.8:
                        inputGame[i][j] = predict[i][j]
            inputGame[int(ind/9)][(ind%9)] = predict[int(ind/9)][(ind%9)]
            inputGame = norm(inputGame)

    return predict



def solve(game):
    game = norm(np.array([int(j) for j in game]).reshape((9,9,1)))
    game = step_by_step(game)
    return game

In [14]:
test_board = '004300209005009001070060043006002087190007400050083000600000105003508690042910300'

solved = solve(test_board)

print(solved)

[[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 5 2 9 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 1 8 3 9 3 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]


In [ ]:
print("End")



